# GIQL operators demo using DuckDB query engine

This notebook demonstrates the 7 core GIQL operators:
- **Binary predicates**: INTERSECTS, WITHIN, CONTAINS
- **Aggregation operators**: MERGE, CLUSTER
- **UDF operators**: DISTANCE
- **Table-valued functions**: NEAREST

For each operator, we:
1. Create a GIQL query
2. Transpile it to standard SQL using GIQL
3. Execute the SQL using DuckDB

In [1]:
import polars as pl
import sqlparse

from giql import GIQLEngine

## Load BED files

Load the two ENCODE ChIP-seq peak files as Polars DataFrames.

In [2]:
# Load BED files with standard BED column names
bed_schema = {
    "chromosome": pl.Utf8,
    "start_pos": pl.Int64,
    "end_pos": pl.Int64,
    "name": pl.Utf8,
    "score": pl.Int64,
    "strand": pl.Utf8,
    "signal_value": pl.Float64,
    "p_value": pl.Float64,
    "q_value": pl.Float64,
    "peak": pl.Int64,
}

# Load first BED file
features_a = pl.read_csv(
    ".ENCFF199YFA.bed", separator="\t", has_header=False, schema=bed_schema
)

# Load second BED file
features_b = pl.read_csv(
    ".ENCFF205OKL.bed", separator="\t", has_header=False, schema=bed_schema
)

print(f"Features A: {len(features_a)} intervals")
print(f"Features B: {len(features_b)} intervals")
print("\nFeatures A preview:")
features_a.head()

Features A: 60893 intervals
Features B: 46196 intervals

Features A preview:


chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
str,i64,i64,str,i64,str,f64,f64,f64,i64
"""chr17""",27381032,27381306,""".""",1000,""".""",461.10586,-1.0,5.08726,136
"""chr7""",25565932,25566199,""".""",1000,""".""",457.60096,-1.0,5.08726,133
"""chr7""",143215117,143215391,""".""",1000,""".""",443.34712,-1.0,5.08726,139
"""chr2""",164600030,164600306,""".""",1000,""".""",442.64446,-1.0,5.08726,137
"""chr15""",56246046,56246359,""".""",1000,""".""",438.56423,-1.0,5.08726,154


## Setup GIQL engine

Create a GIQL engine and register the table schemas.

In [3]:
# Create GIQL engine with DuckDB backend
engine = GIQLEngine(target_dialect="duckdb", verbose=False)

# Register DataFrames using Arrow (preferred for DuckDB)
engine.conn.register("features_a", features_a.to_arrow())
engine.conn.register("features_b", features_b.to_arrow())

# Register schemas with genomic columns
schema_dict = {
    "chromosome": "VARCHAR",
    "start_pos": "BIGINT",
    "end_pos": "BIGINT",
    "name": "VARCHAR",
    "score": "BIGINT",
    "strand": "VARCHAR",
    "signal_value": "DOUBLE",
    "p_value": "DOUBLE",
    "q_value": "DOUBLE",
    "peak": "BIGINT",
}

for table in ["features_a", "features_b"]:
    engine.register_table_schema(table, schema_dict, genomic_column="interval")

print("GIQL engine configured successfully")

GIQL engine configured successfully


---

## 1. INTERSECTS operator

Find features in A that **overlap** with features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval INTERSECTS b.interval`

In [4]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.interval INTERSECTS b.interval
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} overlapping intervals from A")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.*
FROM
  features_a AS a,
  features_b AS b
WHERE (a."chromosome" = b."chromosome"
       AND a."start_pos" < b."end_pos"
       AND a."end_pos" > b."start_pos")


Result: 42616 overlapping intervals from A


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
0,chr11,34359322,34359560,.,1000,.,220.19056,-1.0,5.08726,117
1,chr17,18625675,18625936,.,1000,.,218.02293,-1.0,5.08726,133
2,chr16,67883801,67884000,.,1000,.,219.30369,-1.0,5.08726,102
3,chr14,94036611,94036828,.,1000,.,220.16568,-1.0,5.08726,116
4,chr8,110060410,110060629,.,1000,.,219.42024,-1.0,5.08726,106
5,chr9,135967630,135967829,.,1000,.,218.66102,-1.0,5.08726,92
6,chr14,94136789,94136976,.,1000,.,218.30192,-1.0,5.08726,99
7,chr20,24918423,24918598,.,1000,.,218.92314,-1.0,5.08726,94
8,chr2,202178449,202178624,.,1000,.,217.36688,-1.0,5.08726,90
9,chr7,120987697,120987871,.,1000,.,217.34347,-1.0,5.08726,85


### High overlap depth regions

Find regions with high overlap depth (where many intervals overlap) using a self-join.

In [5]:
# Define GIQL query to find high-depth overlaps
giql_query = """
    WITH overlap_counts AS (
        SELECT 
            a.chromosome,
            a.start_pos,
            a.end_pos,
            a.signal_value,
            COUNT(*) as depth
        FROM features_a a, features_b b
        WHERE a.interval INTERSECTS b.interval
        GROUP BY a.chromosome, a.start_pos, a.end_pos, a.signal_value
    )
    SELECT *
    FROM overlap_counts
    WHERE depth > 2
    ORDER BY depth DESC
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with overlap depth >= 5")
print(f"Max overlap depth: {result['depth'].max() if len(result) > 0 else 0}")
result.head(10)

Transpiled SQL:
WITH overlap_counts AS
  (SELECT
     a.chromosome,
     a.start_pos,
     a.end_pos,
     a.signal_value,
     COUNT(*) AS depth
   FROM
     features_a AS a,
     features_b AS b
   WHERE (a."chromosome" = b."chromosome"
          AND a."start_pos" < b."end_pos"
          AND a."end_pos" > b."start_pos")
   GROUP BY
     a.chromosome,
     a.start_pos,
     a.end_pos,
     a.signal_value)
SELECT *
FROM overlap_counts
WHERE depth > 2
ORDER BY depth DESC


Result: 2 intervals with overlap depth >= 5
Max overlap depth: 3


,chromosome,start_pos,end_pos,signal_value,depth
0,chr14,105999349,105999944,240.51649,3
1,chr14,105999349,105999944,108.06722,3


---

## 2. WITHIN operator

Find features in A that are **completely contained within** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval WITHIN b.interval`

In [6]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*, b.*
    FROM features_a a, features_b b
    WHERE a.interval WITHIN b.interval
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals from A contained within B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT
  a.*,
  b.*
FROM
  features_a AS a,
  features_b AS b
WHERE (a."chromosome" = b."chromosome"
       AND a."start_pos" >= b."start_pos"
       AND a."end_pos" <= b."end_pos")


Result: 30809 intervals from A contained within B


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak,chromosome_1,start_pos_1,end_pos_1,name_1,score_1,strand_1,signal_value_1,p_value_1,q_value_1,peak_1
0,chr14,24216011,24216203,.,1000,.,251.87452,-1.0,5.08726,95,chr14,24215977,24216234,.,1000,.,209.90753,-1.0,5.10518,125
1,chr14,33792757,33792965,.,1000,.,252.08812,-1.0,5.08726,105,chr14,33792735,33793013,.,1000,.,232.27970,-1.0,5.10518,134
2,chr6,43769540,43769748,.,1000,.,250.99653,-1.0,5.08726,104,chr6,43769499,43769760,.,1000,.,154.83893,-1.0,5.10518,141
3,chr17,49861200,49861405,.,1000,.,250.16995,-1.0,5.08726,93,chr17,49861195,49861409,.,1000,.,78.03419,-1.0,5.10518,86
4,chr2,219476345,219476531,.,1000,.,248.18364,-1.0,5.08726,102,chr2,219476292,219476557,.,1000,.,160.44163,-1.0,5.10518,125
5,chr11,57527230,57527419,.,1000,.,247.38897,-1.0,5.08726,104,chr11,57527209,57527456,.,1000,.,121.89660,-1.0,5.10518,134
6,chr2,152025859,152026122,.,1000,.,247.10786,-1.0,5.08726,146,chr2,152025676,152026161,.,1000,.,181.47229,-1.0,5.10518,304
7,chr2,112259523,112259722,.,1000,.,249.77072,-1.0,5.08726,95,chr2,112259464,112259743,.,1000,.,219.16901,-1.0,5.10518,141
8,chr21,43810840,43811034,.,1000,.,201.96817,-1.0,5.08726,102,chr21,43810814,43811039,.,1000,.,98.62279,-1.0,5.10518,131
9,chr17,44331388,44331569,.,1000,.,200.30254,-1.0,5.08726,100,chr17,44331387,44331617,.,1000,.,100.15563,-1.0,5.10518,116


---

## 3. CONTAINS operator

Find features in A that **completely contain** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval CONTAINS b.interval`

In [7]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.interval CONTAINS b.interval
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals from A that contain B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.*
FROM
  features_a AS a,
  features_b AS b
WHERE (a."chromosome" = b."chromosome"
       AND a."start_pos" <= b."start_pos"
       AND a."end_pos" >= b."end_pos")


Result: 4917 intervals from A that contain B


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
0,chr4,102431434,102431659,.,1000,.,299.13707,-1.0,5.08726,111
1,chr14,49586912,49587154,.,1000,.,303.26119,-1.0,5.08726,126
2,chr5,134039564,134039867,.,1000,.,317.39854,-1.0,5.08726,157
3,chr18,77900138,77900404,.,1000,.,314.56958,-1.0,5.08726,127
4,chrX,123960372,123960620,.,1000,.,291.07538,-1.0,5.08726,123
5,chr1,157311233,157311386,.,1000,.,188.57350,-1.0,5.08726,80
6,chr21,32403252,32403390,.,1000,.,132.27311,-1.0,5.08726,80
7,chr14,104564131,104564266,.,1000,.,131.97784,-1.0,5.08726,68
8,chr10,132204836,132204945,.,1000,.,114.77888,-1.0,5.08726,59
9,chrX,54453024,54453130,.,1000,.,102.02848,-1.0,5.08726,65


---

## 4. MERGE operator

**Combine overlapping intervals** from features_a into merged regions.

Similar to `bedtools merge`, this collapses overlapping genomic intervals.

**GIQL Query**: `SELECT MERGE(interval) FROM features_a`

In [8]:
# Define GIQL query - basic merge
giql_query = """
    SELECT MERGE(interval)
    FROM features_b
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals (from {len(features_a)} original)")
result.head(10)

Transpiled SQL:
SELECT
  "chromosome",
  MIN("start_pos") AS start_pos,
  MAX("end_pos") AS end_pos
FROM
  (SELECT
     *,
     SUM(is_new_cluster) OVER (PARTITION BY "chromosome"
                               ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id
   FROM
     (SELECT
        *,
        CASE
            WHEN LAG("end_pos") OVER (PARTITION BY "chromosome"
                                      ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0
            ELSE 1
        END AS is_new_cluster
      FROM features_b) AS lag_calc) AS clustered
GROUP BY
  chromosome,
  __giql_cluster_id
ORDER BY
  "chromosome" NULLS LAST,
  "start_pos" NULLS LAST


Result: 45630 merged intervals (from 60893 original)


,chromosome,start_pos,end_pos
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,913174
5,chr1,919635,919945
6,chr1,931626,931936
7,chr1,938195,938352
8,chr1,951414,951724
9,chr1,976080,976390


### MERGE with aggregation CTE

Compute statistics on merged intervals and filter for intervals that merged multiple features.

In [9]:
# Define GIQL query with aggregations in a CTE
giql_query = """
    WITH merged_intervals AS (
        SELECT
            MERGE(interval),
            COUNT(*) as interval_count,
            AVG(signal_value) as avg_signal
        FROM features_b
    )
    SELECT *
    FROM merged_intervals
    WHERE interval_count > 1
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals with more than 1 original interval")
result.head(10)

Transpiled SQL:
WITH merged_intervals AS
  (SELECT
     "chromosome",
     MIN("start_pos") AS start_pos,
     MAX("end_pos") AS end_pos,
     COUNT(*) AS interval_count,
     AVG(signal_value) AS avg_signal
   FROM
     (SELECT
        *,
        SUM(is_new_cluster) OVER (PARTITION BY "chromosome"
                                  ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id
      FROM
        (SELECT
           *,
           CASE
               WHEN LAG("end_pos") OVER (PARTITION BY "chromosome"
                                         ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0
               ELSE 1
           END AS is_new_cluster
         FROM features_b) AS lag_calc) AS clustered
   GROUP BY
     chromosome,
     __giql_cluster_id
   ORDER BY
     "chromosome" NULLS LAST,
     "start_pos" NULLS LAST)
SELECT *
FROM merged_intervals
WHERE interval_count > 1




Result: 559 merged intervals with more than 1 original interval


,chromosome,start_pos,end_pos,interval_count,avg_signal
0,chr1,1300073,1300592,2,14.844510
1,chr1,8374912,8375330,2,28.608395
2,chr1,9717307,9717819,2,13.833490
3,chr1,14782921,14783438,2,7.107920
4,chr1,18362141,18362677,2,9.312525
5,chr1,19445773,19446283,2,16.590025
6,chr1,23958874,23959385,2,10.616965
7,chr1,24157968,24158482,2,11.985625
8,chr1,27392817,27393359,2,11.789830
9,chr1,30106143,30106664,2,31.694800


### MERGE with distance parameter

Merge intervals within 1000bp of each other.

In [10]:
# Define GIQL query - merge with distance parameter
giql_query = """
    SELECT MERGE(interval, 10000)
    FROM features_b
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals (1000bp distance)")
print(f"Compared to: {len(features_a)} original intervals")
result.head(10)

Transpiled SQL:
SELECT
  "chromosome",
  MIN("start_pos") AS start_pos,
  MAX("end_pos") AS end_pos
FROM
  (SELECT
     *,
     SUM(is_new_cluster) OVER (PARTITION BY "chromosome"
                               ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id
   FROM
     (SELECT
        *,
        CASE
            WHEN LAG("end_pos") OVER (PARTITION BY "chromosome"
                                      ORDER BY "start_pos" NULLS LAST) + 10000 >= "start_pos" THEN 0
            ELSE 1
        END AS is_new_cluster
      FROM features_b) AS lag_calc) AS clustered
GROUP BY
  chromosome,
  __giql_cluster_id
ORDER BY
  "chromosome" NULLS LAST,
  "start_pos" NULLS LAST


Result: 34097 merged intervals (1000bp distance)
Compared to: 60893 original intervals


,chromosome,start_pos,end_pos
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,919945
5,chr1,931626,938352
6,chr1,951414,951724
7,chr1,976080,984477
8,chr1,1013987,1015656
9,chr1,1063787,1064097


---

## 5. CLUSTER operator

**Assign cluster IDs** to overlapping intervals from features_a.

Similar to `bedtools cluster`, this groups overlapping genomic intervals.

**GIQL Query**: `SELECT *, CLUSTER(interval) AS cluster_id FROM features_a`

In [11]:
# Define GIQL query - basic clustering
giql_query = """
    SELECT
        chromosome,
        start_pos,
        end_pos,
        signal_value,
        CLUSTER(interval) AS cluster_id
    FROM features_a
    ORDER BY chromosome, start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with cluster assignments")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT
  chromosome,
  start_pos,
  end_pos,
  signal_value,
  SUM(is_new_cluster) OVER (PARTITION BY "chromosome"
                            ORDER BY "start_pos" NULLS LAST) AS cluster_id
FROM
  (SELECT
     chromosome,
     start_pos,
     end_pos,
     signal_value,
     CASE
         WHEN LAG("end_pos") OVER (PARTITION BY "chromosome"
                                   ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0
         ELSE 1
     END AS is_new_cluster
   FROM features_a) AS lag_calc
ORDER BY
  chromosome,
  start_pos


Result: 60893 intervals with cluster assignments
Number of unique clusters: 5495


,chromosome,start_pos,end_pos,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,2.0
2,chr1,267909,268105,88.00863,3.0
3,chr1,586106,586302,35.02027,4.0
4,chr1,729261,729457,23.29415,5.0
5,chr1,778812,779008,56.97663,6.0
6,chr1,850473,850669,27.08243,7.0
7,chr1,858056,858252,15.55869,8.0
8,chr1,869860,869991,168.87294,9.0
9,chr1,904689,904883,167.56897,10.0


### CLUSTER with distance parameter

Cluster intervals within 1000bp of each other.

In [12]:
# Define GIQL query with distance parameter
giql_query = """
    SELECT
        chromosome,
        start_pos,
        end_pos,
        signal_value,
        CLUSTER(interval, 50000) AS cluster_id
    FROM features_a
    ORDER BY chromosome, start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with cluster assignments (1000bp distance)")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT
  chromosome,
  start_pos,
  end_pos,
  signal_value,
  SUM(is_new_cluster) OVER (PARTITION BY "chromosome"
                            ORDER BY "start_pos" NULLS LAST) AS cluster_id
FROM
  (SELECT
     chromosome,
     start_pos,
     end_pos,
     signal_value,
     CASE
         WHEN LAG("end_pos") OVER (PARTITION BY "chromosome"
                                   ORDER BY "start_pos" NULLS LAST) + 50000 >= "start_pos" THEN 0
         ELSE 1
     END AS is_new_cluster
   FROM features_a) AS lag_calc
ORDER BY
  chromosome,
  start_pos


Result: 60893 intervals with cluster assignments (1000bp distance)
Number of unique clusters: 1376


,chromosome,start_pos,end_pos,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,1.0
2,chr1,267909,268105,88.00863,2.0
3,chr1,586106,586302,35.02027,3.0
4,chr1,729261,729457,23.29415,4.0
5,chr1,778812,779008,56.97663,4.0
6,chr1,850473,850669,27.08243,5.0
7,chr1,858056,858252,15.55869,5.0
8,chr1,869860,869991,168.87294,5.0
9,chr1,904689,904883,167.56897,5.0


---

## 6. DISTANCE operator

**Calculate genomic distances** between intervals from `features_a` and `features_b`.

Similar to `bedtools closest -d`, this calculates the distance in base pairs between genomic intervals.

**GIQL Query**: `SELECT a.*, b.*, DISTANCE(a.interval, b.interval) AS distance FROM features_a a, features_b b`

In [13]:
# Define GIQL query with signed distance
# Signed distance (signed=true) returns negative values for upstream features and positive for downstream
# This is similar to bedtools closest -D ref
giql_query = """
    WITH distances AS (
        SELECT
            a.chromosome,
            a.start_pos AS a_start,
            a.end_pos AS a_end,
            b.start_pos AS b_start,
            b.end_pos AS b_end,
            DISTANCE(a.interval, b.interval, signed=true) AS signed_distance,
            ROW_NUMBER() OVER (
                PARTITION BY a.chromosome, a.start_pos, a.end_pos
                ORDER BY ABS(DISTANCE(a.interval, b.interval, signed=true))
            ) AS rank
        FROM features_a a
        CROSS JOIN features_b b
        WHERE a.chromosome = b.chromosome
            AND a.chromosome = 'chr1'
            AND a.start_pos < 500000
    )
    SELECT 
        chromosome,
        a_start,
        a_end,
        b_start,
        b_end,
        signed_distance,
        CASE 
            WHEN signed_distance < 0 THEN 'upstream'
            WHEN signed_distance > 0 THEN 'downstream'
            ELSE 'overlap'
        END AS direction
    FROM distances
    WHERE rank = 1
    ORDER BY chromosome, a_start
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} features with directional distances")
print(f"Upstream features: {(result['signed_distance'] < 0).sum()}")
print(f"Downstream features: {(result['signed_distance'] > 0).sum()}")
print(f"Overlapping features: {(result['signed_distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
WITH distances AS
  (SELECT
     a.chromosome,
     a.start_pos AS a_start,
     a.end_pos AS a_end,
     b.start_pos AS b_start,
     b.end_pos AS b_end,
     CASE
         WHEN a."chromosome" != b."chromosome" THEN NULL
         WHEN a."start_pos" < b."end_pos"
              AND a."end_pos" > b."start_pos" THEN 0
         WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos")
         ELSE (a."start_pos" - b."end_pos")
     END AS signed_distance,
     ROW_NUMBER() OVER (PARTITION BY a.chromosome, a.start_pos, a.end_pos
                        ORDER BY ABS(CASE
                                         WHEN a."chromosome" != b."chromosome" THEN NULL
                                         WHEN a."start_pos" < b."end_pos"
                                              AND a."end_pos" > b."start_pos" THEN 0
                                         WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos")
                                         E

,chromosome,a_start,a_end,b_start,b_end,signed_distance,direction
0,chr1,181368,181564,186654,186964,5090,downstream
1,chr1,186650,186846,186654,186964,0,overlap
2,chr1,267909,268105,267979,268128,0,overlap


### Signed Distance (Directional)

Calculate **directional distances** where negative values indicate upstream features and positive values indicate downstream features, similar to `bedtools closest -D ref`.

In [14]:
# Define GIQL query - basic distance calculation
# DISTANCE() returns 0 for overlapping intervals, positive integers for gaps between intervals
# Filters to a small region on chr1 for demonstration
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        DISTANCE(a.interval, b.interval) AS distance
    FROM features_a a, features_b b
    WHERE a.chromosome = b.chromosome 
        AND a.chromosome = 'chr1'
        AND a.start_pos BETWEEN 180000 AND 190000
        AND b.start_pos BETWEEN 180000 AND 200000
    ORDER BY a.start_pos, b.start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} distance calculations")
print("\nExample distances (0 = overlap, positive = gap in base pairs):")
result.head(10)

Transpiled SQL:
SELECT
  a.chromosome,
  a.start_pos AS a_start,
  a.end_pos AS a_end,
  b.start_pos AS b_start,
  b.end_pos AS b_end,
  CASE
      WHEN a."chromosome" != b."chromosome" THEN NULL
      WHEN a."start_pos" < b."end_pos"
           AND a."end_pos" > b."start_pos" THEN 0
      WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos")
      ELSE (a."start_pos" - b."end_pos")
  END AS distance
FROM
  features_a AS a,
  features_b AS b
WHERE a.chromosome = b.chromosome
  AND a.chromosome = 'chr1'
  AND a.start_pos BETWEEN 180000 AND 190000
  AND b.start_pos BETWEEN 180000 AND 200000
ORDER BY
  a.start_pos,
  b.start_pos


Result: 2 distance calculations

Example distances (0 = overlap, positive = gap in base pairs):


,chromosome,a_start,a_end,b_start,b_end,distance
0,chr1,181368,181564,186654,186964,5090
1,chr1,186650,186846,186654,186964,0


---

## 7. NEAREST operator

**Find the k-nearest genomic features** using the NEAREST operator.

Similar to `bedtools closest`, this finds the closest features to query intervals. The NEAREST operator eliminates the need to write complex window functions for k-nearest neighbor queries.

**Key Features**:
- Find k-nearest features (k=1 for closest, k>1 for multiple neighbors)
- Directional queries with `signed=true` (upstream vs downstream)
- Distance-constrained queries with `max_distance`
- Strand-specific queries with `stranded=true`
- Implicit reference resolution in LATERAL joins (automatically uses outer table's interval)

**Database Support**:
- **DuckDB**: Full support (LATERAL joins)
- **SQLite**: Standalone mode only (no LATERAL support)

**GIQL Query**: `SELECT * FROM peaks CROSS JOIN LATERAL NEAREST(genes, k=3)`

In [15]:
# Define GIQL query - find closest feature in B for each feature in A
# This replicates bedtools closest -d functionality using NEAREST(target, k=1)
# The reference interval is automatically inferred from the outer table (features_a)
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        a.signal_value AS a_signal,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        b.signal_value AS b_signal,
        distance
    FROM features_a a, NEAREST(features_b, k=1) b
    WHERE a.chromosome = 'chr1'
        AND a.start_pos < 1000000
    ORDER BY a.chromosome, a.start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} features from A with their closest feature in B")
print(f"Average distance to closest feature: {result['distance'].mean():.1f} bp")
print(f"Features overlapping with closest: {(result['distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
SELECT
  a.chromosome,
  a.start_pos AS a_start,
  a.end_pos AS a_end,
  a.signal_value AS a_signal,
  b.start_pos AS b_start,
  b.end_pos AS b_end,
  b.signal_value AS b_signal,
  distance
FROM
  features_a AS a,

  (SELECT
     features_b.*,
     CASE
         WHEN a."chromosome" != features_b."chromosome" THEN NULL
         WHEN a."start_pos" < features_b."end_pos"
              AND a."end_pos" > features_b."start_pos" THEN 0
         WHEN a."end_pos" <= features_b."start_pos" THEN (features_b."start_pos" - a."end_pos")
         ELSE (a."start_pos" - features_b."end_pos")
     END AS distance
   FROM features_b
   WHERE a."chromosome" = features_b."chromosome"
   ORDER BY ABS(CASE
                    WHEN a."chromosome" != features_b."chromosome" THEN NULL
                    WHEN a."start_pos" < features_b."end_pos"
                         AND a."end_pos" > features_b."start_pos" THEN 0
                    WHEN a."end_pos" <= features_b."start_pos" THEN (features_b

,chromosome,a_start,a_end,a_signal,b_start,b_end,b_signal,distance
0,chr1,181368,181564,27.54796,186654,186964,14.61908,5090
1,chr1,186650,186846,19.93098,186654,186964,14.61908,0
2,chr1,267909,268105,88.00863,267979,268128,49.39366,0
3,chr1,586106,586302,35.02027,850424,850734,15.07029,264122
4,chr1,729261,729457,23.29415,850424,850734,15.07029,120967
5,chr1,778812,779008,56.97663,850424,850734,15.07029,71416
6,chr1,850473,850669,27.08243,850424,850734,15.07029,0
7,chr1,858056,858252,15.55869,850424,850734,15.07029,7322
8,chr1,869860,869991,168.87294,869711,870021,12.42156,0
9,chr1,904689,904883,167.56897,912864,913174,31.07865,7981


### Finding k-Nearest Features

Find the **3 closest features** in B for each feature in A using `k=3`.

In [16]:
# Define GIQL query - find 3 nearest features in B for each feature in A
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        a.signal_value AS a_signal,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        b.signal_value AS b_signal,
        distance
    FROM features_a a, NEAREST(features_b, k=3) b
    WHERE a.chromosome = 'chr1'
        AND a.start_pos < 500000
    ORDER BY a.chromosome, a.start_pos, distance
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame

# Group by feature A to show k neighbors per feature
grouped = result.groupby(["a_start", "a_end"]).size()
print(f"Result: {len(result)} total rows (up to 3 neighbors per feature in A)")
print(f"Number of features from A: {len(grouped)}")
print(f"Average neighbors per feature: {grouped.mean():.1f}")
result.head(15)  # Show first 5 features with their 3 neighbors each

Transpiled SQL:
SELECT
  a.chromosome,
  a.start_pos AS a_start,
  a.end_pos AS a_end,
  a.signal_value AS a_signal,
  b.start_pos AS b_start,
  b.end_pos AS b_end,
  b.signal_value AS b_signal,
  distance
FROM
  features_a AS a,

  (SELECT
     features_b.*,
     CASE
         WHEN a."chromosome" != features_b."chromosome" THEN NULL
         WHEN a."start_pos" < features_b."end_pos"
              AND a."end_pos" > features_b."start_pos" THEN 0
         WHEN a."end_pos" <= features_b."start_pos" THEN (features_b."start_pos" - a."end_pos")
         ELSE (a."start_pos" - features_b."end_pos")
     END AS distance
   FROM features_b
   WHERE a."chromosome" = features_b."chromosome"
   ORDER BY ABS(CASE
                    WHEN a."chromosome" != features_b."chromosome" THEN NULL
                    WHEN a."start_pos" < features_b."end_pos"
                         AND a."end_pos" > features_b."start_pos" THEN 0
                    WHEN a."end_pos" <= features_b."start_pos" THEN (features_b

,chromosome,a_start,a_end,a_signal,b_start,b_end,b_signal,distance
0,chr1,181368,181564,27.54796,186654,186964,14.61908,5090
1,chr1,181368,181564,27.54796,267979,268128,49.39366,86415
2,chr1,181368,181564,27.54796,850424,850734,15.07029,668860
3,chr1,186650,186846,19.93098,186654,186964,14.61908,0
4,chr1,186650,186846,19.93098,267979,268128,49.39366,81133
5,chr1,186650,186846,19.93098,850424,850734,15.07029,663578
6,chr1,267909,268105,88.00863,267979,268128,49.39366,0
7,chr1,267909,268105,88.00863,186654,186964,14.61908,80945
8,chr1,267909,268105,88.00863,850424,850734,15.07029,582319


### Directional Queries with signed=true

Find the **3 nearest upstream or downstream features** using `signed=true`.

Negative distances indicate upstream features (B is before A), positive distances indicate downstream features (B is after A).

In [17]:
# Define GIQL query - find 3 nearest features with directional distances
# signed=true returns negative for upstream (B before A) and positive for downstream (B after A)
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        distance,
        CASE
            WHEN distance < 0 THEN 'upstream'
            WHEN distance > 0 THEN 'downstream'
            ELSE 'overlap'
        END AS direction
    FROM features_a a, NEAREST(features_b, k=3, signed=true) b
    WHERE a.chromosome = 'chr1'
        AND a.start_pos < 500000
        AND direction = 'upstream'
    ORDER BY a.chromosome, a.start_pos, ABS(distance)
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame

# Analyze directionality
upstream = (result["distance"] < 0).sum()
downstream = (result["distance"] > 0).sum()
overlap = (result["distance"] == 0).sum()

print(f"Result: {len(result)} total nearest features")
print(f"Upstream features (distance < 0): {upstream}")
print(f"Downstream features (distance > 0): {downstream}")
print(f"Overlapping features (distance = 0): {overlap}")
result.head(15)

Transpiled SQL:
SELECT
  a.chromosome,
  a.start_pos AS a_start,
  a.end_pos AS a_end,
  b.start_pos AS b_start,
  b.end_pos AS b_end,
  distance,
  CASE
      WHEN distance < 0 THEN 'upstream'
      WHEN distance > 0 THEN 'downstream'
      ELSE 'overlap'
  END AS direction
FROM
  features_a AS a,

  (SELECT
     features_b.*,
     CASE
         WHEN a."chromosome" != features_b."chromosome" THEN NULL
         WHEN a."start_pos" < features_b."end_pos"
              AND a."end_pos" > features_b."start_pos" THEN 0
         WHEN a."end_pos" <= features_b."start_pos" THEN (features_b."start_pos" - a."end_pos")
         ELSE (a."start_pos" - features_b."end_pos")
     END AS distance
   FROM features_b
   WHERE a."chromosome" = features_b."chromosome"
   ORDER BY ABS(CASE
                    WHEN a."chromosome" != features_b."chromosome" THEN NULL
                    WHEN a."start_pos" < features_b."end_pos"
                         AND a."end_pos" > features_b."start_pos" THEN 0
         

,chromosome,a_start,a_end,b_start,b_end,distance,direction


### Distance-Constrained Queries with max_distance

Find up to **5 nearest features within 50kb** using `max_distance=50000`.

This is useful for finding nearby regulatory elements or associated genes within a biologically relevant distance.

In [18]:
# Define GIQL query - find up to 5 nearest features within 50kb
# max_distance=1000000 filters out features farther than 50kb
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        distance
    FROM features_a a, NEAREST(features_b, k=5, max_distance=1000000) b
    WHERE a.chromosome = 'chr1'
        AND a.start_pos < 500000
    ORDER BY a.chromosome, a.start_pos, distance
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame

# Analyze distance constraints
grouped = result.groupby(["a_start", "a_end"]).size()
print(
    f"Result: {len(result)} total rows (up to 5 neighbors within 50kb per feature in A)"
)
print(f"Number of features from A: {len(grouped)}")
print(f"Average neighbors per feature: {grouped.mean():.1f}")
print(f"Max distance found: {result['distance'].max()} bp (should be <= 50000)")
print(f"Features with at least one neighbor: {len(grouped)}")
result.head(15)

Transpiled SQL:
SELECT
  a.chromosome,
  a.start_pos AS a_start,
  a.end_pos AS a_end,
  b.start_pos AS b_start,
  b.end_pos AS b_end,
  distance
FROM
  features_a AS a,

  (SELECT
     features_b.*,
     CASE
         WHEN a."chromosome" != features_b."chromosome" THEN NULL
         WHEN a."start_pos" < features_b."end_pos"
              AND a."end_pos" > features_b."start_pos" THEN 0
         WHEN a."end_pos" <= features_b."start_pos" THEN (features_b."start_pos" - a."end_pos")
         ELSE (a."start_pos" - features_b."end_pos")
     END AS distance
   FROM features_b
   WHERE a."chromosome" = features_b."chromosome"
     AND (ABS(CASE
                  WHEN a."chromosome" != features_b."chromosome" THEN NULL
                  WHEN a."start_pos" < features_b."end_pos"
                       AND a."end_pos" > features_b."start_pos" THEN 0
                  WHEN a."end_pos" <= features_b."start_pos" THEN (features_b."start_pos" - a."end_pos")
                  ELSE (a."start_pos" - fe

,chromosome,a_start,a_end,b_start,b_end,distance
0,chr1,181368,181564,186654,186964,5090
1,chr1,181368,181564,267979,268128,86415
2,chr1,181368,181564,850424,850734,668860
3,chr1,181368,181564,869711,870021,688147
4,chr1,181368,181564,912864,913174,731300
5,chr1,186650,186846,186654,186964,0
6,chr1,186650,186846,267979,268128,81133
7,chr1,186650,186846,850424,850734,663578
8,chr1,186650,186846,869711,870021,682865
9,chr1,186650,186846,912864,913174,726018


---

## Summary

This demo showcased all 7 GIQL operators:

1. **INTERSECTS**: Binary predicate for overlapping intervals
2. **WITHIN**: Binary predicate for containment (A within B)
3. **CONTAINS**: Binary predicate for containment (A contains B)
4. **MERGE**: Aggregation operator to combine overlapping intervals
5. **CLUSTER**: Aggregation operator to assign cluster IDs to overlapping intervals
6. **DISTANCE**: UDF operator to calculate genomic distances between intervals
7. **NEAREST**: Table-valued function for finding k-nearest genomic features

Each operator was:
- Written in GIQL syntax
- Transpiled to standard SQL
- Executed using DuckDB

This demonstrates how GIQL provides a high-level, intuitive syntax for genomic interval operations while maintaining compatibility with standard SQL engines through transpilation.

In [19]:
# Clean up
# engine.close()

In [20]:
import polars as pl

# Load first BED file to check dtypes
bed_schema = {
    "chromosome": pl.Utf8,
    "start_pos": pl.Int64,
    "end_pos": pl.Int64,
    "name": pl.Utf8,
    "score": pl.Int64,
    "strand": pl.Utf8,
    "signal_value": pl.Float64,
    "p_value": pl.Float64,
    "q_value": pl.Float64,
    "peak": pl.Int64,
}

features_a = pl.read_csv(
    ".ENCFF199YFA.bed", separator="\t", has_header=False, schema=bed_schema
)

# Check pandas conversion
adf = features_a.to_pandas()
print("Pandas dtypes:")
print(adf.dtypes)
print("\nFirst few rows:")
print(adf.head(2))

Pandas dtypes:
chromosome          str
start_pos         int64
end_pos           int64
name                str
score             int64
strand              str
signal_value    float64
p_value         float64
q_value         float64
peak              int64
dtype: object

First few rows:
  chromosome  start_pos   end_pos name  score strand  signal_value  p_value  \
0      chr17   27381032  27381306    .   1000      .     461.10586     -1.0   
1       chr7   25565932  25566199    .   1000      .     457.60096     -1.0   

   q_value  peak  
0  5.08726   136  
1  5.08726   133  
